## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nador,MA,35.1740,-2.9287,21.00,46,0,6.17,clear sky
1,1,Lorengau,PG,-2.0226,147.2712,26.14,85,100,3.73,overcast clouds
2,2,Bangassou,CF,4.7413,22.8184,34.15,32,100,1.48,overcast clouds
3,3,Kiama,AU,-34.6833,150.8667,19.44,82,1,0.45,clear sky
4,4,Sept-Iles,CA,50.2001,-66.3821,-3.33,58,75,4.12,broken clouds
...,...,...,...,...,...,...,...,...,...,...
691,691,Necochea,AR,-38.5473,-58.7368,18.89,85,52,2.68,broken clouds
692,692,Pokhara,NP,28.2333,83.9833,20.59,10,0,2.62,clear sky
693,693,Vila,PT,42.0304,-8.1588,20.00,59,0,3.58,clear sky
694,694,Merrill,US,45.1805,-89.6835,3.33,74,1,1.34,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 10
What is the maximum temperature you would like for your trip? 25


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.tail()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
688,688,Porto Walter,BR,-8.2686,-72.7439,23.21,97,100,1.15,light rain
691,691,Necochea,AR,-38.5473,-58.7368,18.89,85,52,2.68,broken clouds
692,692,Pokhara,NP,28.2333,83.9833,20.59,10,0,2.62,clear sky
693,693,Vila,PT,42.0304,-8.1588,20.00,59,0,3.58,clear sky
695,695,Damietta,EG,31.4165,31.8133,19.00,55,20,4.63,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                231
City                   231
Country                231
Lat                    231
Lng                    231
Max Temp               231
Humidity               231
Cloudiness             231
Wind Speed             231
Current Description    231
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

#preferred_cities_df.dropna()
clean_df = preferred_cities_df

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Nador,MA,21.00,clear sky,35.1740,-2.9287,
3,Kiama,AU,19.44,clear sky,-34.6833,150.8667,
5,Mataura,NZ,14.44,broken clouds,-46.1927,168.8643,
11,Rikitea,PF,24.83,light rain,-23.1203,-134.9692,
15,Port Alfred,ZA,21.32,scattered clouds,-33.5906,26.8910,


In [8]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}

In [9]:
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    # Add latitude and longitude to parameters for search
    params["location"] = f"{lat},{lng}"
    # Create base_url for search
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data
    hotels = requests.get(base_url, params=params).json()
    # Store name of first hotel in hotel DF
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError): 
        print("No hotel found...skipping.")

No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.


In [10]:
hotel_df["Lat"] = hotel_df["Lat"].map("{:,.2f}".format)
hotel_df["Lng"] = hotel_df["Lng"].map("{:,.2f}".format)
hotel_df


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Nador,MA,21.00,clear sky,35.17,-2.93,Hôtel La Giralda
3,Kiama,AU,19.44,clear sky,-34.68,150.87,The Sebel Kiama Harbourside
5,Mataura,NZ,14.44,broken clouds,-46.19,168.86,Ellie's Villa
11,Rikitea,PF,24.83,light rain,-23.12,-134.97,Pension Maro'i
15,Port Alfred,ZA,21.32,scattered clouds,-33.59,26.89,The Halyards Hotel
...,...,...,...,...,...,...,...
688,Porto Walter,BR,23.21,light rain,-8.27,-72.74,Secretaria de saúde
691,Necochea,AR,18.89,broken clouds,-38.55,-58.74,dyd -mar Hotel
692,Pokhara,NP,20.59,clear sky,28.23,83.98,Butterfly Lodge
693,Vila,PT,20.00,clear sky,42.03,-8.16,Hotel Castrum Villae by Walk Hotels


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace([np.inf, -np.inf], np.nan, inplace=True)



In [12]:
hotel_df.replace([np.inf, -np.inf], np.nan, inplace=True)
hotel_df.dropna(inplace=True)
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Nador,MA,21.00,clear sky,35.17,-2.93,Hôtel La Giralda
3,Kiama,AU,19.44,clear sky,-34.68,150.87,The Sebel Kiama Harbourside
5,Mataura,NZ,14.44,broken clouds,-46.19,168.86,Ellie's Villa
11,Rikitea,PF,24.83,light rain,-23.12,-134.97,Pension Maro'i
15,Port Alfred,ZA,21.32,scattered clouds,-33.59,26.89,The Halyards Hotel


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °C</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
#hotel_info


In [15]:
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
hotel_df['Lat'] = pd.to_numeric(hotel_df['Lat'])
hotel_df['Lng'] = pd.to_numeric(hotel_df['Lng'])

In [16]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

# 